In [1]:
import numpy as np
import random
import itertools
import pandas as pd
import multiprocessing, threading
import math
import scipy.stats as ss
import time
import pickle
import re

from opyenxes.model.XLog import XLog
from opyenxes.data_in.XUniversalParser import XUniversalParser
from opyenxes.classification.XEventAttributeClassifier import XEventAttributeClassifier

from prefixspan import PrefixSpan

from tqdm import *

data_path = "../logs/bpic2011.xes"
traces_picklepath = data_path.replace(".xes", "_raw_traces.pickled")
traces_tmppath = data_path.replace(".xes", "_traces_tmp.pickled")
traces_finalpath = data_path.replace(".xes", "_traces_encoded.pickled")

In [ ]:
with open(data_path) as bpic_file:
    eventlog = XUniversalParser().parse(bpic_file)[0]

In [2]:
ncores = multiprocessing.cpu_count()
ntraces = len(eventlog)

NameError: name 'eventlog' is not defined

## Extract data trace-wise from XES format and enrich with BOS/EOS markers

In [ ]:
# collect all attributes
column_names = []

for event in eventlog[0]:
    for attribute in event.get_attributes():
        column_names.append(attribute)
        
column_names = set(column_names) # remove duplicates
column_names = list(column_names)

def create_dataframe_from_trace(t):
    df = pd.DataFrame(columns=column_names, index=range(0,len(t)))
    for event_idx, event in enumerate(t):
        event_attributes = event.get_attributes()
        df.iloc[event_idx]["__case_id"] = 0
        
        for attribute in event_attributes:
            df[attribute].values[event_idx] = event_attributes[attribute].get_value()
    
    return df

ppool = multiprocessing.Pool(ncores)
traces = []
with tqdm(total=len(eventlog), desc="Converting XES traces to Pandas dataframes", unit="traces") as pbar:
    for i, _ in tqdm(enumerate(ppool.imap(create_dataframe_from_trace, eventlog))):
        pbar.update()
        traces.append(_)
        
del eventlog

In [ ]:
pickle.dump(traces, open(traces_picklepath, "wb"))

In [3]:
traces = pickle.load(open(traces_picklepath, "rb"))

## Eliminate correlated or unimportant features

In [ ]:
def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
        https://stackoverflow.com/questions/46498455/categorical-features-correlation"""
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

for col_a,col_b in itertools.product(eventlog_df.columns, repeat=2): 
    candidate = pd.crosstab(eventlog_df[col_a], eventlog_df[col_b]).as_matrix()
    print("{: >30} {: >30} {: >20}".format(col_a, col_b, cramers_v(candidate)))

In [4]:
# lifecyle:transition is always "complete"
# Producer code correlates perfectly with org:group
# Activity code correlates perfectly with concept:name
for t in traces:
    t.drop(columns=["lifecycle:transition", "Producer code", "Activity code", "Section"], inplace=True)

## Create standard feature set

In [5]:
eventlog_df = pd.concat(traces, ignore_index=True)

## Create SP2 feature set

In [ ]:
# https://stackoverflow.com/questions/42636765/how-to-set-all-the-values-of-an-existing-pandas-dataframe-to-zero
# This one-hot encodes all entries in concept:name column for later incrementation once it has been seen
# sp2_features = pd.get_dummies(eventlog_df["concept:name"], prefix="SP2") # can't use windowed representation here as it might skew distribution of values
# eventlog_sp2_df = process_results.copy(deep=True)
# sp2_features    = sp2_features.drop(sp2_features.index[sp2_features.index[len(eventlog_sp2_df):]])
# assert(len(sp2_features) == len(eventlog_sp2_df))

In [6]:
# loop through every trace and encode the presence of an activity
sp2_prefix = "SP2_"
activity_labels = [ "{0}{1}".format(sp2_prefix,a) for a in eventlog_df["concept:name"].unique() ]

def enrich_trace_with_sp2(t):
    sp2_df = pd.DataFrame(columns=activity_labels, index=range(0,len(t)), dtype=np.bool)
    for col in sp2_df.columns: sp2_df[col].values[:] = 0
    sp2_df["{0}{1}".format(sp2_prefix, t["concept:name"][0])].values[0]  = 1
    
    for i in range(1,len(t)):
        first_activity_name = t["concept:name"].iloc[i]
        col = "{0}{1}".format(sp2_prefix,first_activity_name)
        
        sp2_df.values[i] = sp2_df.values[i-1]
        sp2_df[col].values[i] = 1
        
    return sp2_df

ppool = multiprocessing.Pool(ncores)
sp2_traces = []
with tqdm(total=len(traces), desc="Enriching traces with SP2 features", unit="traces") as pbar:
    for i, _ in tqdm(enumerate(ppool.imap(enrich_trace_with_sp2, traces))):
        pbar.update()
        sp2_traces.append(_)

Enriching traces with SP2 features:  71%|███████   | 808/1143 [00:03<00:01, 237.92traces/s]
808it [00:03, 215.51it/s]
Enriching traces with SP2 features:  94%|█████████▍| 1080/1143 [00:04<00:00, 274.09traces/s]
1093it [00:04, 284.44it/s]
Enriching traces with SP2 features: 100%|██████████| 1143/1143 [00:04<00:00, 263.36traces/s]


## Enrich with PrefixSpan features

In [7]:
def print_patterns(pt):
    for p in pt:
        print("Support: {0}%".format(100*p[0]/len(traces)))
        for n in p[1]:
            print("    > ", int_to_event[n])
        print()

# since most patterns begin and end with the <eos> and <bos> markers, the features only become valuable towards the end...
events       = eventlog_df["concept:name"].unique()
event_to_int = dict((c, i) for i,c in enumerate(events) if c not in ["<bos>","<eos>"])
int_to_event = dict((i, c) for i,c in enumerate(events) if c not in ["<bos>","<eos>"])

In [ ]:
traces = save_traces[:]

In [8]:
# Prefixspan requires an array of arrays with one subarray for every trace
encoded_traces = [ t["concept:name"].map(event_to_int).tolist() for t in traces ]
prefixspan_traces = PrefixSpan(encoded_traces)
closed_sequences = prefixspan_traces.topk(25, closed=True) # support is how often the subsequence appears in total
# http://sequenceanalysis.github.io/slides/analyzing_sequential_user_behavior_part2.pdf, slide 5
# print_patterns(ps_topkc)

# only take subsequence which are at a certain level of support? like if ss[0]/len(traces) < .90
#ps_topkc = list(filter(lambda x: x[0]/len(traces) > .90, ps_topkc))
closed_sequences = [ p[1] for p in closed_sequences ]
pftrace_args = [ (t, closed_sequences[:], event_to_int) for t in traces ] # enrich traces with copy of mined subsequences

In [9]:
def wrapped__enrich_trace_with_subseq(args):
    return enrich_trace_with_subseq(*args)

def enrich_trace_with_subseq(t, ps, event_to_int):
    col_prefix = "PFS_"
    subseq_labels = [ "{0}{1}".format(col_prefix,ss_idx) for ss_idx, ss in enumerate(ps) ]
    subseq_df = pd.DataFrame(columns=subseq_labels, index=range(0,len(t)), dtype=np.bool)
    
    for col in subseq_df.columns: subseq_df[col].values[:] = 0
    for i in range(0,len(t)): # loop through sequence, prune items from mined sequences, and once a subsequence array is empty, this subsequence has occured :)
        activity_code = event_to_int.get(t["concept:name"].iloc[i], None)
        
        for subseq_idx in range(0,len(ps)):
            if ps[subseq_idx] == []:
                continue
            if ps[subseq_idx][0] == activity_code:
                ps[subseq_idx].pop(0)
                if ps[subseq_idx] == []:
                    subseq_df.values[i:,subseq_idx] = 1
        
    return subseq_df

ppool = multiprocessing.Pool(ncores)
pf_traces = []

with tqdm(total=len(pftrace_args), desc="Enriching traces with mined subsequence features", unit="traces") as pbar:
    for i, _ in tqdm(enumerate(ppool.imap(wrapped__enrich_trace_with_subseq, pftrace_args))):
        pbar.update()
        pf_traces.append(_)

Enriching traces with mined subsequence features:   0%|          | 0/1143 [00:00<?, ?traces/s]
Enriching traces with mined subsequence features:   1%|          | 13/1143 [00:00<00:10, 104.05traces/s]
Enriching traces with mined subsequence features:   6%|▋         | 73/1143 [00:00<00:07, 135.04traces/s]
Enriching traces with mined subsequence features:  12%|█▏        | 132/1143 [00:00<00:05, 172.43traces/s]
Enriching traces with mined subsequence features:  18%|█▊        | 210/1143 [00:00<00:04, 224.09traces/s]
Enriching traces with mined subsequence features:  24%|██▍       | 279/1143 [00:00<00:03, 280.82traces/s]
Enriching traces with mined subsequence features:  29%|██▉       | 333/1143 [00:00<00:02, 309.48traces/s]
Enriching traces with mined subsequence features:  36%|███▌      | 414/1143 [00:00<00:01, 379.17traces/s]
Enriching traces with mined subsequence features:  41%|████      | 471/1143 [00:00<00:01, 421.04traces/s]
Enriching traces with mined subsequence features:  46%|████

## Do basic encoding of remaining features

In [10]:
# Convert timestamp to total running time
bos_idx = 0
for i in range(0, len(traces)):
    tlen = len(traces[i])-1
    dfs = traces[i]["time:timestamp"] - traces[i]["time:timestamp"][bos_idx]
    traces[i]["time:timestamp"] = dfs.map(lambda d: int(d.total_seconds()/3600)) # convert to hours

In [11]:
# finally produce TARGET feature
eventlog_df["TARGET"] = eventlog_df["concept:name"].shift(-1)
target_offset = 0

for t in traces:
    target_offset += len(t)
    eventlog_df["TARGET"].values[target_offset-1] = "<EOS>"

In [12]:
import copy
save_traces = copy.deepcopy(traces)

In [20]:
traces = copy.deepcopy(save_traces)

In [13]:
# Do one-hot encoding for concept:name, org:group and TARGET
dummy_columns = ["concept:name", "org:group"]
eventlog_df_dummies = pd.get_dummies(eventlog_df[dummy_columns], dtype=np.bool)
target_df_dummies = pd.get_dummies(eventlog_df["TARGET"], prefix="TARGET", dtype=np.bool)
trace_offset = 0

for i in range(0,len(traces)):
    tlen = len(traces[i])
    onehot_features = eventlog_df_dummies[trace_offset:trace_offset+tlen].reset_index(drop=True)
    onehot_targets  = target_df_dummies[trace_offset:trace_offset+tlen].reset_index(drop=True)
               
    traces[i].drop(columns=dummy_columns, inplace = True, errors = 'ignore')
    traces[i] = pd.concat([traces[i], onehot_features, sp2_traces[i], pf_traces[i], onehot_targets], ignore_index=False, axis=1)
    trace_offset += tlen

In [14]:
# Set data types for columns that were not autodetected
for i in range(0,len(traces)):
    traces[i]["Specialism code"] = pd.to_numeric(traces[i]["Specialism code"], errors="ignore")
    traces[i]["Number of executions"] = pd.to_numeric(traces[i]["Number of executions"], errors="ignore")

In [26]:
assert(sum([len(t) for t in traces]) == len(eventlog_df))
pickle.dump(traces, open(traces_finalpath, "wb"), protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
used_bytes = sum([t.memory_usage(index=True, deep=True).sum() for t in traces])
print("Memory usage of encoded data: {0} KB".format(used_bytes / 1024))
print("Memory usage of encoded data: {0} MB".format(used_bytes / 1024**2))
print("Memory usage of encoded data: {0} GB".format(used_bytes / 1024**3))

In [ ]:
trace_columns = traces[0].columns.tolist()
trace_columns = list(map(lambda e: bool(re.match('^TARGET_.+', e)), trace_columns))
trace_columns.index(True) # when do the target columns start?!